<a href="https://colab.research.google.com/github/kaluznys/uczenie_maszynowe_UW/blob/main/praca_domowa6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Understanding Splitting Criteria in CART for Regression**
---------------------

In this assignment, you will explore three common formulations of the splitting criterion used in **CART (Classification and Regression Trees)** for **regression problems**:

1. **Local RSS Minimization**  
2. **RSS Gain Maximization**  
3. **Total RSS Minimization**

You will investigate whether any of these criteria are equivalent, and you will design an experiment to determine which criterion is actually employed in a standard implementation such as **scikit-learn’s DecisionTreeRegressor**.



## **The Problem**

Many treatments of CART for regression describe the split selection process in different ways. Below are three frequently cited formulations. Suppose we have a dataset with features $X$ and target $y$, and we seek to choose a feature $X_j$ and a threshold $t$ to split the data into two regions $R_1(X_j, t)$ and $R_2(X_j, t)$. Denote by $\bar{y}_{R_m}$ the mean of targets within region $R_m$.

1. **Local RSS Minimization**  
   We select the feature and threshold that minimize the **sum of squared errors** in the two resulting child nodes:
   $$
   (X_j^*, t^*) = \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
   $$

2. **RSS Gain Maximization**  

   It is also a local method, looking only at a parent and two child nodes.

   We select the feature and threshold that maximize the **reduction** in RSS, computed by subtracting the RSS of the two child nodes from the RSS in the parent node:
   $$
   (X_j^*, t^*) = \arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\}.
   $$

3. **Total RSS Minimization**  
   For a dataset $\{(x_i, y_i)\}_{i=1}^N$ with features $X$ and target $y$, let $T$ be the current tree.

   For any split on feature $X_j$ at threshold $t$, define $T(X_j, t)$ as the new tree obtained by splitting one leaf of $T$ into two leaves $R_1(X_j, t)$ and $R_2(X_j, t)$.
   
   Let $\mathrm{Leaves}(T(X_j, t))$ be the set of all leaf indices in this new tree. For each leaf $m \in \mathrm{Leaves}(T(X_j, t))$, define:
   $$
   R_m = \{\, i \,\mid\, x_i \text{ ends in leaf } m\}.
   $$

   $R_m$ set collects all data indices $i$ whose feature vector $x_i$ is classified into the leaf node $m$ when passed through the tree $T(X_j,t)$. In other words, each leaf node $m$ in $T(X_j, t)$ corresponds to a unique path of splits, and any data point $x_i$ that follows that path is assigned to the leaf $m$; hence, it belongs to $R_m$.

   $R_m$ sets for all leafs $m \in \mathrm{Leaves}(T(X_j, t))$ define a partition of all indices.

   Then the objective of **minimizing total Residual Sum of Squares (total RSS)** is stated as:
   $$
   (X_j^*, t^*) = \arg\min_{(X_j, t)} \sum_{m \in \mathrm{Leaves}(T(X_j, t))}
   \sum_{i \in R_m} \Bigl(y_i - \overline{y}_{R_m}\Bigr)^2,
   $$
   where
   $$
   \overline{y}_{R_m} = \frac{1}{\lvert R_m \rvert}
   \sum_{i \in R_m} y_i
   $$
   is the mean response in leaf $m$.


## **Research Questions**

1. **Equivalence Analysis**  
   Determine whether the above formulations are equivalent or if they can yield different split choices. Specifically:
   - Are *local RSS minimization* and *RSS gain maximization* equivalent?
   - Does *total RSS minimization* coincide with either of these two, or is it distinct?
   
2. **Empirical Experiment**  
   Design and conduct a Python experiment to determine which of these formulations is implemented in `scikit-learn` in `DecisionTreeRegressor`. Present numerical results and plots to support your conclusion.


## **Tasks & Deliverables**

1. **Formulation Analysis**  
   - Compare *local RSS minimization*, *RSS gain maximization*, and *total RSS minimization*.
   - If you find that any pair of formulations is equivalent, provide a concise proof.  
   - If you find that they differ, construct a counterexample.

2. **Empirical Verification**  
   - Create a small artificial dataset and train a `DecisionTreeRegressor` from `scikit-learn`.
   - The dataset must be designed in a way that uniquely identifies the formulation used. Provide a short code snippet and a plot or table to support your conclusion.

3. **Report**  
   - Summarize your theoretical insights and empirical findings in a **Colab notebook**.
   - Include the relevant proofs, code, discussion, and conclusions.
   - Place the notebook in your **GitHub repository** for this course, add a link to it in your README.md and add an **“Open in Colab”** badge in the notebook so it can be launched directly.



In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [61]:
np.random.seed(42)
n_samples=200
ages = np.random.randint(20, 70, size=n_samples)
incomes = np.random.normal(loc=60000, scale=15000, size=n_samples).astype(int)
house_sizes = np.random.normal(loc=50, scale=15, size=n_samples).astype(int)
nr_of_houses = np.random.poisson(lam=1, size=n_samples)
nr_of_houses  = np.ceil(nr_of_houses)
locations = np.random.choice(['urban', 'suburban', 'rural'], size=n_samples)

In [62]:
df = pd.DataFrame({
    'Age': ages,
    'Income': incomes,
    'HouseSize': house_sizes,
    'NrOfHouses': nr_of_houses,
    'Location': locations,
})
df = pd.get_dummies(df, "location")

In [68]:
house_prices = (
    50000 +
    (ages * -100) +
    (incomes * 0.5) +
    (house_sizes * 4800) +
    (nr_of_houses * 10000) +
    (df.location_rural*-50000) +
    (df.location_suburban*30000)+
    (df.location_urban*-30000) +
    np.random.normal(0, 10000, size=n_samples)
)
df['price'] = house_prices
df.head()

,Age,Income,HouseSize,NrOfHouses,location_rural,location_suburban,location_urban,price
0,58,57580,49,0.0,False,True,False,317180.811385
1,48,66060,51,0.0,False,False,True,322407.528472
2,34,88292,69,0.0,False,False,True,383234.662360
3,62,62618,41,0.0,False,True,False,305942.900234
4,27,63863,58,0.0,False,True,False,378246.820898


In [1]:
def rss(y):
    if len(y) == 0:
        return 0
    return np.sum((y - np.mean(y)) ** 2)

In [3]:
class TreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, prediction=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.prediction = prediction

In [86]:
def best_split_rss_gain(X, y):
    best_feature, best_value = None, None
    best_rss = float('inf')

    for feature_idx in range(X.shape[1]):
        values = np.unique(X[:, feature_idx])

        for val in values:
            left_mask = X[:, feature_idx] <= val
            right_mask = ~left_mask

            y_left, y_right = y[left_mask], y[right_mask]

            rss_gain = rss(y_left) + rss(y_right)  # Local RSS

            if rss_gain > best_rss:
                best_rss = rss_gain
                best_feature = feature_idx
                best_value = val

    return best_feature, best_value, best_rss

In [35]:
def build_tree_rss_gain(X, y, max_depth=3, depth=0):
    if depth >= max_depth or len(y) <= 5:
        return TreeNode(prediction=np.mean(y))

    feature, value, split_rss_gain = best_split_rss_gain(X, y)

    if feature is None:
        return TreeNode(prediction=np.mean(y))

    left_mask = X[:, feature] <= value
    right_mask = ~left_mask

    left = build_tree_rss_gain(X[left_mask], y[left_mask], max_depth, depth + 1)
    right = build_tree_rss_gain(X[right_mask], y[right_mask], max_depth, depth + 1)

    return TreeNode(feature=feature, value=value, left=left, right=right)

In [36]:
def predict_one(x, node):
    while node.prediction is None:
        if x[node.feature] <= node.value:
            node = node.left
        else:
            node = node.right
    return node.prediction

def predict(X, tree):
    return np.array([predict_one(x, tree) for x in X])

In [82]:
df_nump = df.to_numpy()
X_train, y_train, X_test, y_test = df_nump[:150, :-1], df_nump[:150, -1], df_nump[:50, :-1], df_nump[:50, -1]

In [46]:
X_train

array([[58, 57580, 49, 0.0, False, False, True],
       [48, 66060, 51, 0.0, False, True, False],
       [34, 88292, 69, 0.0, False, True, False],
       [62, 62618, 41, 0.0, True, False, False],
       [27, 63863, 58, 0.0, False, True, False],
       [40, 58883, 46, 0.0, True, False, False],
       [58, 31218, 46, 0.0, False, False, True],
       [38, 59602, 66, 1.0, True, False, False],
       [42, 60903, 62, 1.0, False, False, True],
       [30, 96948, 62, 1.0, True, False, False],
       [30, 57114, 69, 0.0, False, True, False],
       [43, 64523, 50, 3.0, True, False, False],
       [55, 59479, 60, 0.0, True, False, False],
       [59, 42469, 45, 2.0, False, False, True],
       [43, 77142, 54, 1.0, False, True, False],
       [22, 71278, 48, 1.0, True, False, False],
       [41, 71865, 51, 0.0, False, True, False],
       [21, 46359, 58, 0.0, True, False, False],
       [43, 81041, 37, 1.0, True, False, False],
       [63, 38972, 81, 0.0, False, True, False],
       [49, 68802, 3

In [83]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((150, 7), (150,), (50, 7), (50,))

In [87]:
tree = build_tree_rss_gain(X_train, y_train, max_depth=3)
y_pred = predict(X_train, tree)
mse = mean_squared_error(y_train, y_pred)
round(mse**(1/2), 0)

76587.0

In [54]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
round(mse**(1/2), 0)

53916.0

In [85]:
y_pred

array([311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 311136.98396812, 311136.98396812, 311136.98396812,
       311136.98396812, 3